In [7]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

# Module 2: 

_Contents_
* [Timing](#sec_timing)
* Intersymbol Interference


## Timing <a class="anchor" id="sec_timing"></a>

In [8]:
class GateBase(object):
  def __init__(self, t_prop_delay=1e-9, initial_state=False):
    self._current_output_state = initial_state
    self._next_output_state = initial_state # Will move to current after tpd.
    self._time_of_last_state_change = 0
    self._tpd = t_prop_delay
    self._input_states = [] # length of list defined by *argv.

  def get_state(self, t):
    self._update_state(t)
    return int(self._current_output_state)

  def set_input(self, t, *argv):
    if len(self._input_states) != len(argv):
      raise ValueError("Lengths of input argv and input states differ.")

    bool_argv = [bool(in_) for in_ in argv] 
    
    # Check if any input states have changed.
    for ii, new_input_i in enumerate(argv):
      if self._input_states[ii] != new_input_i:
        self._time_of_last_state_change = t # reset the timer
        self._input_states[ii] = new_input_i
    
    self._update_next_output_state()

  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    raise NotImplementedError("Derived class should implement this function.")

  def _update_state(self, t):
    if t - self._time_of_last_state_change >= self._tpd:
      self._current_output_state = self._next_output_state
      #self._time_of_last_state_change = t # reset the timer
    

class NAND2Gate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=True):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False, False]
    self._update_next_output_state()

  def set_input(self, t, *argv):
    super().set_input(t, *argv)
    
  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    a = self._input_states[0]
    b = self._input_states[1]
    self._next_output_state = not ( a and b )

class NOTGate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=False):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False,]
    self._update_next_output_state()

  def _update_next_output_state(self):
    a = self._input_states[0]
    self._next_output_state = not a

## NAND Gate Testing
Test out the functionality of a NAND2 gate on its own before integrating it with other logic.

In [9]:
tpd = 4

t = np.linspace(0,50,101)

G1 = NAND2Gate(tpd)

a = 1*(t > 6) - 1*(t > 33)
b = 1*(t > 11) - 1*(t > 20)

y = 0*t
for ii, v_ in enumerate(zip(t, a, b)):
  t_, a_, b_ = v_
  G1.set_input(t_, a_, b_)
  y[ii] = G1.get_state(t_)

xfcn = lambda x_ : x_*1
fig, ax = plt.subplots(1,1)
ax.plot(xfcn(t), a, label='a')
ax.plot(xfcn(t), b+1, label='b')
ax.plot(xfcn(t), y+2, label='y')
ax.grid(True)
ax.set_title('NAND2 Testing')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Logic Level')
ax.legend(loc='best')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## SR Latch Testing
The SR latch uses two NAND2 gates with feedback from the outputs.  In certain transitions the gates may reach a metastable point where the result is not obvious due to process variation or, in an ideal case, oscillates.

In [10]:
tpd = 4

t = np.linspace(0,100,100001)

G1 = NAND2Gate(tpd, initial_state=False)
G2 = NAND2Gate(tpd, initial_state=True)

#s = 1 - (1*(t >= 10) - 1*(t >= 20) + 1*(t >= 70))
#r = 1 - (1*(t >= 40) - 1*(t >= 50))
s = 1*(t >= 10) - 1*(t >= 25) + 1*(t >=50)
r = 1*(t >= 15) - 1*(t >= 30) + 1*(t >=50)
#s = 1 - 1*(t > 40)
#r = 1 + 0*t


q = 0*t
qn = 1 + 0*t
for ii, t_ in enumerate(t):
  if ii == 0:
    q_, qn_ = G1.get_state(t_), G2.get_state(t_)
  else:
    q_, qn_ = q[ii-1], qn[ii-1]
  s_, r_ = s[ii], r[ii]

  G1.set_input(t_, s_, qn_)
  G2.set_input(t_, r_, q_)
  q[ii] = G1.get_state(t_)
  qn[ii] = G2.get_state(t_)
  

xfcn = lambda x_ : x_*1
fig, ax = plt.subplots(1,1)
ax.plot(xfcn(t), s, label='$\overline{S}$')
ax.plot(xfcn(t), r+1, label='$\overline{R}$')
ax.plot(xfcn(t), qn+2, label='~q', color='C2', linestyle='--')
ax.plot(xfcn(t), q+3, label='q', color='C2')
ax.grid(True)
leg1 = ax.legend(loc='best')

ax.set_title('SR Latch Testing')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Logic Level')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Logic Level')

In [11]:
tpd = 2.5

t = np.linspace(0,200,100001)

# D Latch Specific Gates
G0 = NOTGate(tpd, initial_state=True)
G3 = NAND2Gate(tpd, initial_state=True)
G4 = NAND2Gate(tpd, initial_state=False)

# SR Latch Gates
G1 = NAND2Gate(tpd, initial_state=False)
G2 = NAND2Gate(tpd, initial_state=True)

Td = 40
D = 1*((t % Td) > Td/2)
#D = 1*((t >= 20)&(t < 20+Td/2))

T = 50
CLK = 1*((t % T) >= T/2)
#CLK = 1*((t >= 40)
#s = 1 - 1*(t > 40)
#r = 1 + 0*t

s = 0*t
r = 0*t
Dn = 0*t
q = 0*t
qn = 1 + 0*t
for ii, t_ in enumerate(t):
  if ii == 0:
    qn_ = G1.get_state(t_)
    q_ = G2.get_state(t_)
    Dn_ = G0.get_state(t_)
    s_ = G3.get_state(t_)
    r_ = G4.get_state(t_)
  else:
    q_, qn_ = q[ii-1], qn[ii-1]
    s_, r_ = s[ii-1], r[ii-1]
    Dn_ = Dn[ii-1]
  D_, CLK_ = D[ii], CLK[ii]
  

  G0.set_input(t_, D_)
  G3.set_input(t_, D_, CLK_)
  G4.set_input(t_, Dn_, CLK_)
  G1.set_input(t_, s_, qn_)
  G2.set_input(t_, r_, q_)

  Dn[ii] = G0.get_state(t_)
  s[ii] = G3.get_state(t_)
  r[ii] = G4.get_state(t_)
  q[ii] = G1.get_state(t_)
  qn[ii] = G2.get_state(t_)

xfcn = lambda x_ : x_*1
fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.plot(xfcn(t), D+6, label='D')
ax.plot(xfcn(t), CLK+5, label='CLK')
ax.plot(xfcn(t), Dn+4, label='Dn')
ax.plot(xfcn(t), s+3, label='$\overline{S}$')
ax.plot(xfcn(t), r+2, label='$\overline{R}$')
ax.plot(xfcn(t), q+1, label='q', color='C5')
ax.plot(xfcn(t), qn, label='~q', color='C5', linestyle='--')

ax.grid(True)
leg1 = ax.legend(loc='best')

ax.set_title('SR Latch Testing')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Logic Level')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Logic Level')

In [12]:
tpd = 2.5

t = np.linspace(0,200,100001)

# D Latch Specific Gates
G0 = NOTGate(tpd, initial_state=True)
G3 = NAND2Gate(tpd, initial_state=True)
G4 = NAND2Gate(tpd, initial_state=False)

# SR Latch Gates
G1 = NAND2Gate(tpd, initial_state=False)
G2 = NAND2Gate(tpd, initial_state=True)

Td = 40
D = 1*((t % Td) > Td/2)
#D = 1*((t >= 20)&(t < 20+Td/2))

T = 50
CLK = 1*((t % T) >= T/2)
#CLK = 1*((t >= 40)
#s = 1 - 1*(t > 40)
#r = 1 + 0*t

s = 0*t
r = 0*t
Dn = 0*t
q = 0*t
qn = 1 + 0*t
for ii, t_ in enumerate(t):
  if ii == 0:
    qn_ = G1.get_state(t_)
    q_ = G2.get_state(t_)
    Dn_ = G0.get_state(t_)
    s_ = G3.get_state(t_)
    r_ = G4.get_state(t_)
  else:
    q_, qn_ = q[ii-1], qn[ii-1]
    s_, r_ = s[ii-1], r[ii-1]
    Dn_ = Dn[ii-1]
  D_, CLK_ = D[ii], CLK[ii]
  

  G0.set_input(t_, D_)
  G3.set_input(t_, D_, CLK_)
  G4.set_input(t_, Dn_, CLK_)
  G1.set_input(t_, s_, qn_)
  G2.set_input(t_, r_, q_)

  Dn[ii] = G0.get_state(t_)
  s[ii] = G3.get_state(t_)
  r[ii] = G4.get_state(t_)
  q[ii] = G1.get_state(t_)
  qn[ii] = G2.get_state(t_)

xfcn = lambda x_ : x_*1
fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.plot(xfcn(t), D+6, label='D')
ax.plot(xfcn(t), CLK+5, label='CLK')
ax.plot(xfcn(t), Dn+4, label='Dn')
ax.plot(xfcn(t), s+3, label='$\overline{S}$')
ax.plot(xfcn(t), r+2, label='$\overline{R}$')
ax.plot(xfcn(t), q+1, label='q', color='C5')
ax.plot(xfcn(t), qn, label='~q', color='C5', linestyle='--')

ax.grid(True)
leg1 = ax.legend(loc='best')

ax.set_title('SR Latch Testing')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Logic Level')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Logic Level')